In [1]:
import numpy as np
import open3d as o3d
from scipy.spatial import KDTree
import meshplot as mp
from util import Interpolation

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
folder_dataset = '../data/generated/'
folder_gen = 'test2'

In [3]:
def plotCenterline( centerline , p ):
    shading = { 'point_size': 0.1 }
    p.add_points( np.array([centerline[0]]), shading=shading)

    for i in range(1, len(centerline)):
        p.add_points( np.array([centerline[i]]), shading=shading )
        p.add_lines(  np.array([centerline[i-1]]), np.array([centerline[i]]), shading=shading )  

In [4]:
def normalize( mesh: o3d.geometry.TriangleMesh, centerline: np.ndarray ) -> None:
    #minCoord = np.min( np.asarray(mesh.vertices) )
    #mesh.translate( (- minCoord, - minCoord, - minCoord )) #ahora deberia estar en rango [0,0,0] -> [inf, inf, inf] cada vertice
    #centerline += np.full_like(centerline, -minCoord)
#
    #maxCoord = np.max( np.asarray(mesh.vertices) )
    #mesh.scale( 2/maxCoord, center=(0,0,0)) # ahora en rango [0,0,0], [2,2,2]
    #centerline *= 2/maxCoord
#
    #mesh.translate( (- 1, - 1, - 1 )) # ahora en rango [-1,-1,-1], [1,1,1]
    #centerline += np.full_like(centerline, -1)

    centro = mesh.get_center()
    mesh.translate( (0,0,0), relative=False)
    centerline -= np.full_like(centerline, centro)

    maxCoord = np.max( np.abs( np.asarray(mesh.vertices) ) )

    mesh.scale( 1/maxCoord, center=(0,0,0)) # ahora en rango [0,0,0], [2,2,2]
    centerline *= 1/maxCoord

In [5]:
original = o3d.io.read_triangle_mesh( folder_dataset + 'test1.off' )
centerline = np.load(folder_dataset + 'test1.npy')

mesh= o3d.geometry.TriangleMesh(original)
normalize( mesh, centerline )

print( np.min( np.asarray(mesh.vertices) ), np.max(np.asarray(mesh.vertices) ))

p = mp.plot( np.asarray(mesh.vertices), np.asarray( mesh.triangles ), c = np.array([0.7,0,0]), return_plot=True )
#p.add_mesh( np.asarray(original.vertices), np.asarray( original.triangles ), c = np.array([0,0,0.7]) )
p.add_lines( np.array([[0,0,0], [0,0,0], [0,0,0] ]), np.array([[1,0,0], [0,1,0], [0,0,1] ]))
plotCenterline( centerline, p)


-1.0 0.9999077913221517


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-4.610419…

In [6]:
p.add_points( np.asarray( mesh.vertices ), shading={'point_size': 0.08, 'point_color':'blue'})

11

In [7]:
treeCenters = KDTree( centerline )

a = treeCenters.query( np.asarray( mesh.vertices ), k=1 )

In [8]:
submeshes = { 0: [], 1: [], 2: [], 3: [], 4: []}

for idx, (point, nn) in enumerate( zip(np.asarray( mesh.vertices ), a[1] ) ):
    submeshes[nn].append( idx )

In [9]:
r = [len(submeshes[i] ) for i in range(5)]
print( r, len(mesh.vertices), sum(r))

[144, 256, 256, 256, 144] 1056 1056


In [10]:
recs = []
for m in submeshes.values():
    newMesh = o3d.geometry.TriangleMesh( mesh  )
    newMesh.remove_vertices_by_mask( [ i not in m for i in range(len(mesh.vertices)) ])
    recs.append( newMesh )

In [11]:
p3 = mp.plot( np.zeros((1,3)) , return_plot=True)
colors = np.array([[1,0,0], [0,1,0], [0,0,1], [1,1,0], [0, 1, 1]])
for m, col in zip(recs, colors):
    p3.add_mesh( np.asarray(m.vertices), np.asarray(m.triangles), c=col)

plotCenterline( centerline, p3 )

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [12]:
curva = Interpolation( centerline )
bases = curva.basisAlong( np.linspace(0,1,5) )

idx = 1

copia = o3d.geometry.TriangleMesh(recs[idx])
copia.translate( -centerline[idx] )

B_inv = np.linalg.inv(bases[idx].T)
T = np.block( [[ B_inv, np.zeros((3,1)) ], [np.zeros((1,3)), np.array([1]) ] ])

copia.transform( T )

ccopia = o3d.geometry.TriangleMesh(copia)
normalize( ccopia, ccopia.get_center() )

p4 = mp.plot( np.asarray(copia.vertices), np.asarray(copia.triangles), c=np.array([0.7,0,0]), return_plot=True)
p4.add_mesh( np.asarray(recs[idx].vertices), np.asarray(recs[idx].triangles), c=np.array([0,0,0.7]))
p4.add_mesh( np.asarray(ccopia.vertices), np.asarray(ccopia.triangles), c=np.array([0,0.7,0.0]))
p4.add_lines( np.array([[0,0,0], [0,0,0], [0,0,0] ]), np.array([[1,0,0], [0,1,0], [0,0,1] ]))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.006820…

3

In [13]:
import os
os.mkdir('../data/segmented/' + folder_gen )
os.mkdir('../data/segmented/' + folder_gen +'/originals')
os.mkdir('../data/segmented/' + folder_gen +'/euclidean')
os.mkdir('../data/segmented/' + folder_gen +'/normalized')

curva = Interpolation( centerline )
bases = curva.basisAlong( np.linspace(0,1,len(centerline)) )

for i, (smesh, base) in enumerate(zip(recs, bases)):
    o3d.io.write_triangle_mesh(f'../data/segmented/{folder_gen}/originals/test1_{i}.ply', smesh)

    norm = o3d.geometry.TriangleMesh(smesh)
    norm.translate( -centerline[i] )

    B_inv = np.linalg.inv(base.T) # existe porque son ortogonales entre si -> son li
    T = np.block( [[ B_inv, np.zeros((3,1)) ], [np.zeros((1,3)), np.array([1]) ] ])

    norm.transform( T )
    o3d.io.write_triangle_mesh(f"../data/segmented/{folder_gen}/euclidean/test1_{i}.ply", norm)

    normalize(norm, norm.get_center())
    o3d.io.write_triangle_mesh(f"../data/segmented/{folder_gen}/normalized/test1_{i}.ply", norm)
